In [1]:
#########################
#########  
######### 
#########################
import xarray as xr
import numpy as np
import datetime
import pandas

import cartopy
import cartopy.feature as cfeat
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable




In [2]:
#########################
######### DICTIONARIES DEFINITION 
#########################

reg_ext = {
    'lab': {
        'lon' : (-60, -30),
        'lat' : (50, 65),
        'name' : 'Labrador and Irminger Seas'
    },
    'gul': {
        'lon' : (-75, -45),
        'lat' : (30, 45),
        'name' : 'Gulf Stream'
    },
    'noe': {
        'lon' : (-30, -5),
        'lat' : (45, 60),
        'name' : 'North East Sea'
    }
}

In [3]:
def Grid(data_set):
    """
        Grid is a function that creates a rectangular grid using as x a longitude
        array and for y a latitudde array.
        
        Parameters:
        ------------
            
        data_set : DataArray
            Is the dataset from which we will plot the histogram.
        
        Output:
        -------
        (x, y) : n-arrays
            Arrays that correspond for each (lon,lat) point
    """
    x = data_set.longitude
    y = data_set.latitude
    
    x, y = np.meshgrid(x, y)
    return(x, y)


## Function to crop the dataset

def Crops(coord, d_set):
    """
        Crops is a function that takes a data set and crops it into smaller
        regions, using as parameters the values given by the dictionary 
        reg_ext.
        
        Parameters:
        ------------
            
        coord : string
            Key value that identifies the region to obtain
        
        d_set : DataArray
            Dataset to be cropped
        
        Output:
        -------
        new_ds : DataArray
            New data array corresponding to the region stated by 'coord'
    """
    
    lon1, lon2 = reg_ext[coord]['lon']
    lat1, lat2 = reg_ext[coord]['lat']
    name_fig = reg_ext[coord]['name']
    
    new_ds = d_set.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))

    return(new_ds)


def Histogram(data_set, n_bins, xlims=None, ylims=None, i=None, ax=None):
    """
        Histogram is a function that helps to make a semi-log histogram plot
        of a dataset. The 'y' axis is logaritmic, and the 'x' axis is linear.
        The function accepts a dataset with any kind of values and it filters the
        nan values.
        
        Parameters:
        ------------
            
        data_set : DataArray
            Is the dataset from which we will plot the histogram.
        
        n_bins : integer
            Number of bins for the histogram.
            
        xlims : tuple, float
            The limits for the x axis
        
        ylims : tuple, float
            The limits for the y axis
        
        i : integer
            Is the time step we adre working on. if None, it returns the
            complete array's histogram
            
        ax : axes.Axes object or array of Axes objects
            axes of the n-th sub plot
        
        Output:
        -------
        Plot, Fig.
    """
    ## Creation of an array to save all values for the histogram
    ### This could be done in a faster way!!
    
    a = []
    
    if i == None:
        for j in range(len(data_set[:])):
            a = np.append(a, data_set[j])
    else:
        for j in range(len(data_set[i])):
            a = np.append(a, data_set[i, j])

    ##Taking away the nan values
    a2 = No_nan(a)
    ran = (xlims)
        
    if not ax:
        ax = plt.gca()
    
    ax.hist(a2, n_bins, range=ran, alpha=0.75)
    ax.set_yscale("log", nonposy='clip')
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.grid(True)
    
    
def No_nan(a):
    """
        No_nan is a function that helps to filter an array from nan values.
        
        Parameters:
        ------------
        a : Numpy Array
            Is the array we want to filter
        
        Output:
        -------
        a2 : Numpy Arrray
            Array with no nan values in it
    """
    nan_array = np.isnan(a)
    not_nan_array = ~ nan_array
    a2 = a[not_nan_array]
    
    return(a2)

In [ ]:
### Here, trying to make a more general function to make the animations

In [4]:
dir_1 = '/home/lgarcia/Documents/data_ARMOR/'
dir_2 = '/net/alpha/exports/sciences/data/LPO_ISAS/ANA_ISAS15/fld2D/'

fl_n1 = 'ARMOR_*.nc'
fl_n2 = 'ISAS15_DM_2006_2015_MLDS.nc'

c_armor = xr.open_mfdataset(dir_1 + fl_n1)
mld_a = c_armor.mlotst

monthly_avg = mld_a.resample(time="1MS").mean('time')

c_isas = xr.open_mfdataset(dir_2 + fl_n2)
mld_i = c_isas.MLDP




In [ ]:
coord = 'noe'
reg_noeA = Crops(coord, mld_a)
reg_noeI = Crops(coord, mld_i)
xlims = 10, 1800
ylims = 10e-2, 10e6
t = ':'

# Embedded histogram 

By now just putting two plots together

In [ ]:
dir_1 = '/home/lgarcia/Documents/data_ARMOR/'
fl_n1 = 'ARMOR_*.nc'
c_armor = xr.open_mfdataset(dir_1 + fl_n1)
mld_a = c_armor.mlotst

list_month = [6, 7, 8, 9, 10, 11]

time = pandas.to_datetime(mld_a[it].time.values)
month = time.month

coord = 'noe'
reg = Crops(coord, mld_a)

n_bins = 100
xlim = 10., 1800.
ylim = 10e-1, 10e4


%matplotlib notebook

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(9,7), sharex=True)
Histogram(reg_noeA, 100, xlims, ylims, None, ax[0])
ax[0].set_title('ARMOR')

plt.ylabel('# of occuped grid points')
plt.title(time)
plt.hlines(120*60, *xlim)
plt.show()



Histogram(reg_noeI, 100, xlims, ylims, None, ax[1])
ax[1].set_title('ISAS')

plt.xlabel('MLD (m)')
plt.ylabel('# of occuped grid points')
#plt.title(time)
plt.hlines(120*60, *xlim)
fig.suptitle(' ', str(time))
plt.show()




In [ ]:
if month in list_month:
    xlim2 = 9., 250.
    ylim2 = 10e-1, 10e4

    # this is an inset axes over the main axes
    a = plt.axes([.45, .65, .42, .2])
    Histogram(reg, n_bins, xlim2, ylim2, it)
    plt.xticks()
    plt.yticks()


In [ ]:
## Aca tratando lo de Anne Marie

In [9]:
mld_a[0].values

array([[ 19.899902,  19.899902,  19.899902, ...,        nan,        nan,
               nan],
       [ 19.899902,  19.899902,  19.899902, ...,        nan,        nan,
               nan],
       [ 20.      ,  20.      ,  21.699951, ...,        nan,        nan,
               nan],
       ...,
       [ 20.      ,  20.      ,  20.      , ..., 120.      , 107.59985 ,
        111.      ],
       [ 20.      ,  20.      ,  20.      , ..., 115.19995 , 116.69995 ,
        105.19995 ],
       [ 20.      ,  20.      ,  20.      , ..., 110.69995 , 188.09985 ,
        106.59985 ]], dtype=float32)

array([[[ 19.899902 ,  19.899902 ,  19.899902 , ...,   0.       ,
           0.       ,   0.       ],
        [ 19.899902 ,  19.899902 ,  19.899902 , ...,   0.       ,
           0.       ,   0.       ],
        [ 20.       ,  20.       ,  21.699951 , ...,   0.       ,
           0.       ,   0.       ],
        ...,
        [ 20.       ,  20.       ,  20.       , ..., 120.       ,
         107.59985  , 111.       ],
        [ 20.       ,  20.       ,  20.       , ..., 115.19995  ,
         116.69995  , 105.19995  ],
        [ 20.       ,  20.       ,  20.       , ..., 110.69995  ,
         188.09985  , 106.59985  ]],

       [[ 20.       ,  20.       ,  19.899902 , ...,   0.       ,
           0.       ,   0.       ],
        [ 22.300049 ,  22.5      ,  20.       , ...,   0.       ,
           0.       ,   0.       ],
        [ 50.       ,  51.30005  ,  52.80005  , ...,   0.       ,
           0.       ,   0.       ],
        ...,
        [ 20.       ,  20.       ,  20.       , ..., 1